In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../src/')  
from scraper.session import login, BASE_URL
from scraper.fetch import get_court_links
# from scraper.parser import parse_cases
from scraper.court_scraper import CourtScraper



# getting court links doesnt need session as no log in... but it does want you to log in to view the deetts..
links = get_court_links()
# print(links)

# logging in
session = login()      


# parse cases using session data from login
# parse_cases(session, links)
for link in links:
    courtScraoer = CourtScraper(session, BASE_URL + link)
    courtScraoer.load_case_page()
    courtScraoer.get_case_list_soup()
    courtScraoer.extract_city_and_court_name()
    court_cases = courtScraoer.rows_to_objects()
    if not court_cases:
        continue
    for case in court_cases:
        print(case)







getting court links
username: analogue1@live.com, password: xtmpw463
Status Code: 302
Final URL: https://courtserve.net/
session cookies: <RequestsCookieJar[<Cookie SITELOKPW=cjV6cEpPQldGNDlRK3BHcWhiYXFNdjZscUZJQ0cwRytTcG5TdkVvPQ%3D%3D for .courtserve.net/>, <Cookie PHPSESSID=4660fad1bf86709b394ad6ed2730c3d6fa81e59a for courtserve.net/>]>
probs logged in
 extracted Aldershot & Farnham from the court name
CourtCase(case_id='AF25F00026', start_time_string='10:00 AM', duration='30 minutes', claimant='TURNER', defendant='HUMPHREY', hearing_type='Family Law Injunction (Return)', hearing_channel='In Person')
CourtCase(case_id='AF25F00028', start_time_string='2:00 PM', duration='30 minutes', claimant='PATON', defendant='RANKIN', hearing_type='Family Law Injunction (Return)', hearing_channel='In Person')
 extracted Barnet from the court name
CourtCase(case_id='BT24F00101', start_time_string='10:00 AM', duration='5\r\n    hours', claimant='P -', defendant='- S', hearing_type='Final Hearing (Fam

TypeError: 'NoneType' object is not iterable